In [694]:
%matplotlib inline

In [695]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
import obspy
import os
from obspy.clients.fdsn import Client
from obspy.clients.syngine import Client as SyngineClient
from obspy.taup import TauPyModel
from obspy.geodetics import kilometer2degrees
from obspy.io.xseed import Parser
from obspy.signal.cross_correlation import xcorr,xcorr_3c

# Coletando os dados das estações BP

In [696]:
STA = ['BDCO','BPPF','BUCO','GENI','GRJU','PRDT','STSN','STSR','TRZN']
z = 8

In [697]:
filter_folder = 'a_25'

In [698]:
ev_list = []
ev_listS = []

for root, dirs, files in os.walk('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/rede_BB/event_data_PP_selected/'+filter_folder+'/'+STA[z]):
    for datafile in files:
        if datafile.endswith('.eqr'):
            ev_list.append(os.path.join(root, datafile))
ev_listS = sorted(ev_list)

In [699]:
ev = obspy.Stream()
for i,j in enumerate(ev_listS):
    ev += obspy.read(j,headonly=True)

In [700]:
event_DD = []
event_MM = []
event_YYYY = []
event_hh = []
event_mm = []
event_julday = []
event_depth = []
event_lat = []
event_long = []
event_dist = []
event_sta = []
event_channel = []
sta_lat = []
sta_long = []
sta_channel = []
event_starttime = []
event_endtime = []


for i,j in enumerate(ev):
    event_time = (j.stats.starttime)
    event_starttime.append(j.stats.starttime)
    event_endtime.append(j.stats.endtime)
    event_DD.append("{0:02.0f}".format(event_time.day))
    event_MM.append("{0:02.0f}".format(event_time.month))
    event_YYYY.append(event_time.year)
    event_hh.append("{0:02.0f}".format(event_time.hour))
    event_mm.append("{0:02.0f}".format(event_time.minute))
    event_julday.append(event_time.julday)
    event_depth.append(j.stats.sac.evdp)
    event_lat.append(j.stats.sac.evla)
    event_long.append(j.stats.sac.evlo)
    event_dist.append(j.stats.sac.dist)
    event_sta.append(j.stats.station)
    sta_lat.append(j.stats.sac.stla)
    sta_long.append(j.stats.sac.stlo)

In [701]:
eventid_lst = []
for k,l in enumerate(ev):
    eventid_lst.append('GCMT:C'+str(event_YYYY[k])+str(event_MM[k])+str(event_DD[k])+str(event_hh[k])+str(event_mm[k])+'A')

eventid_lst = sorted(list(set(eventid_lst)))

In [702]:
initial_time = [str(i) for i in event_starttime]
initial_time = sorted(list(set(initial_time)))
starttime_lst = [obspy.UTCDateTime(i) for i in initial_time] 

In [703]:
lat_lst = list(set(sta_lat))
long_lst = list(set(sta_long))
sta_lst = list(set(event_sta))

## Calculando os sismogramas sintéticos através do servidor da Syngine IRIS

In [704]:
%%time

c_s = SyngineClient()
#model = ["ak135f_2s", "iasp91_2s", "prem_a_2s","prem_i_2s"]
model = "iasp91_2s"


st_synth = []
for k,l in enumerate(eventid_lst):
    try:
        print('Evento '+l)
        st_synth.append(obspy.Stream(c_s.get_waveforms(model=model, 
                                                receiverlatitude=lat_lst[0],
                                                receiverlongitude=long_lst[0],
                                                networkcode='BP',
                                                stationcode=sta_lst[0],
                                                eventid=eventid_lst[k],
                                                dt="0.01",units="displacement",starttime="P-10", endtime="P+260",
                                                format='saczip')))
    except:
        print('Problema no Evento: '+eventid_lst[k])

Evento GCMT:C201605180757A
Evento GCMT:C201605181646A
Evento GCMT:C201605280946A
Problema no Evento: GCMT:C201605280946A
Evento GCMT:C201606071051A
Evento GCMT:C201606100325A
Evento GCMT:C201607251726A
Evento GCMT:C201608190732A
Evento GCMT:C201608210345A
Evento GCMT:C201608240136A
Evento GCMT:C201610300640A
Evento GCMT:C201611080455A
Evento GCMT:C201611202057A
Evento GCMT:C201611241843A
Evento GCMT:C201612251422A
Evento GCMT:C201704031740A
Evento GCMT:C201704242138A
Evento GCMT:C201705102323A
Evento GCMT:C201706140729A
Evento GCMT:C201706221231A
Evento GCMT:C201707202231A
Evento GCMT:C201709080449A
Evento GCMT:C201709191814A
Evento GCMT:C201710101853A
CPU times: user 692 ms, sys: 138 ms, total: 831 ms
Wall time: 2min 19s


In [705]:
loc_folder = '/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/syngine_PP/'+filter_folder+'/'

In [706]:
%%time

for k,l in enumerate(st_synth):
    for r,t in enumerate(l):
        evdp = t.stats.sac.evdp/1000
        gcarc = t.stats.sac.gcarc
            #Calculando o parâmetro do raio
        model = TauPyModel(model="iasp91")
        arrivals = model.get_travel_times(source_depth_in_km=evdp, distance_in_degree=gcarc, phase_list=["P"])
        arr = arrivals[0]
        j = t.stats.starttime
        t.stats.sac.user8 = arr.ray_param/6371
        folder_loc_string = loc_folder+t.stats.station+'/'+str(j.year)+'/'+str("{0:0=3d}".format(j.julday))+'/'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)
        os.makedirs(folder_loc_string,exist_ok=True)
        t.write(folder_loc_string+'/SYN.'+t.stats.network+'.'+t.stats.station+'.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+t.stats.channel[-1],format='SAC')

CPU times: user 3.83 s, sys: 27.2 ms, total: 3.86 s
Wall time: 3.86 s
